In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/wn-dataset-prediction/__results__.html
/kaggle/input/wn-dataset-prediction/__output__.json
/kaggle/input/wn-dataset-prediction/custom.css
/kaggle/input/wn-dataset-prediction/__notebook__.ipynb
/kaggle/input/wn-dataset-prediction/df.csv
/kaggle/input/wn-dataset-prediction/__results___files/__results___20_0.png
/kaggle/input/wn-dataset-prediction/__results___files/__results___14_1.png


In [13]:
from sklearn.model_selection import train_test_split

import category_encoders as ce 
from sklearn.pipeline import Pipeline

import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from matplotlib import pyplot

from sklearn.ensemble import StackingRegressor

In [14]:
df=pd.read_csv("/kaggle/input/wn-dataset-prediction/df.csv")

In [15]:
X_1=df[['MONTH','ORIGIN_AIRPORT','DESTINATION_AIRPORT','DISTANCE']]
y_1=df['ARRIVAL_DELAY']
X=X_1.to_numpy()
y=y_1.to_numpy()

In [16]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('knn', KNeighborsRegressor()))
    level0.append(('cart', DecisionTreeRegressor()))
    
    # define meta learner model
    level1 = LinearRegression()
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=2)
    return model

In [17]:
# get a list of models to evaluate
def get_models():
    models = dict()
    models['knn'] = KNeighborsRegressor()
    models['cart'] = DecisionTreeRegressor()
    models['stacking'] = get_stacking()
    return models

In [18]:
def evaluate_model(model):
    cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [19]:
models = get_models()

In [20]:
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))

>knn -21.067 (0.113)
>cart -18.868 (0.043)


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [ ]:
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()